In [1]:
from mhkit import dolfyn as dlfn
from dolfyn.adp import api
from mhkit.dolfyn.rotate import api as ap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from numpy.fft import fft, ifft
from datetime import datetime
import xarray as xr
import scipy.interpolate as interp
import utm
from mhkit.river.io import d3d 
from os.path import abspath, dirname, join, normpath, relpath
import netCDF4
plt.rcParams.update({'font.size': 15}) # Set font size of plots title and labels 

C:\Users\eabrow\AppData\Roaming\Python\Python38\site-packages\dolfyn\velocity.py:10: AccessorRegistrationWarning: registration of accessor <class 'dolfyn.velocity.Velocity'> under name 'velds' for type <class 'xarray.core.dataset.Dataset'> is overriding a preexisting attribute with the same name.
  class Velocity():


In [2]:
ds_02 = api.read('data/river/ADCP/tanana_transects_08_10_10_0_002_10-08-10_142214.PD0')
ds_03 = api.read('data/river/ADCP/tanana_transects_08_10_10_0_003_10-08-10_143335.PD0')

ds_02_03= xr.merge([ds_02,ds_03])
ds_02_03


Reading file data/river/ADCP/tanana_transects_08_10_10_0_002_10-08-10_142214.PD0 ...
  Encountered end of file.  Cleaning up data.

Reading file data/river/ADCP/tanana_transects_08_10_10_0_003_10-08-10_143335.PD0 ...
  Encountered end of file.  Cleaning up data.


<xarray.Dataset>
Dimensions:              (time: 753, dir: 4, beam: 4, range: 47, x: 4, x*: 4, earth: 3, inst: 3)
Coordinates:
  * time                 (time) datetime64[ns] 2010-08-10T14:28:15.559999942 ...
  * dir                  (dir) <U3 'X' 'Y' 'Z' 'err'
  * beam                 (beam) int32 1 2 3 4
  * range                (range) float64 0.57 0.82 1.07 ... 11.57 11.82 12.07
  * x                    (x) int32 1 2 3 4
  * x*                   (x*) int32 1 2 3 4
  * earth                (earth) <U1 'E' 'N' 'U'
  * inst                 (inst) <U1 'X' 'Y' 'Z'
Data variables: (12/24)
    number               (time) float64 3.652e+03 3.654e+03 ... 4.904e+03
    builtin_test_fail    (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    c_sound              (time) float32 1.466e+03 1.466e+03 ... 1.466e+03
    depth                (time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    pitch                (time) float32 -0.1 -0.21 -0.21 -0.21 ... 0.2 0.07 0.56
    roll                 (time) float32 3.33 3.33 3.49 3.82 ... 2.24 2.65 2.33
    ...                   ...
    vel_bt               (dir, time) float32 0.063 0.039 0.06 ... 0.007 0.01 0.0
    corr_bt              (dir, time) float64 231.0 242.0 237.0 ... 249.0 246.0
    amp_bt               (dir, time) float64 40.0 36.0 39.0 ... 40.0 64.0 49.0
    prcnt_gd_bt          (dir, time) float64 0.0 100.0 0.0 ... 100.0 100.0 100.0
    beam2inst_orientmat  (x, x*) float64 1.462 -1.462 0.0 ... -1.034 -1.034
    orientmat            (earth, inst, time) float64 -0.9037 -0.9064 ... 1.0 1.0
Attributes: (12/38)
    inst_make:                TRDI
    inst_model:               Workhorse
    inst_type:                ADCP
    rotate_vars:              ['vel', 'vel_bt']
    has_imu:                  0
    name:                     unrecognized firmware version
    ...                       ...
    xmit_pulse:               0.3
    water_ref_cells:          [0, 5]
    fls_target_threshold:     50
    xmit_lag_m:               0.08
    fs:                       11.11111111111111
    vel_gps_corrected:        0

In [3]:
data=utm.from_latlon(ds_02_03.latitude_gps, ds_02_03.longitude_gps, 5, 'W')
latitude=data[0]
longitude=data[1]

AttributeError: 'Dataset' object has no attribute 'latitude_gps'

In [4]:
# Downloading Data
datadir = normpath(join(relpath(join('data', 'river', 'd3d'))))
filename= 'tanana50_map.nc'
d3d_data = netCDF4.Dataset(join(datadir,filename)) 

# Printing variable and description
for var in d3d_data.variables.keys():
    try: 
        d3d_data[var].long_name
    except:
        print(f'"{var}"')        
    else:
        print(f'"{var}": {d3d_data[var].long_name}')

"mesh2d_enc_x": x-coordinate
"mesh2d_enc_y": y-coordinate
"mesh2d_enc_node_count": count of coordinates in each instance geometry
"mesh2d_enc_part_node_count": count of nodes in each geometry part
"mesh2d_enc_interior_ring": type of each geometry part
"mesh2d_enclosure_container"
"Mesh2D"
"NetNode_x": x-coordinate
"NetNode_y": y-coordinate
"projected_coordinate_system"
"NetNode_z": bed level at net nodes (flow element corners)
"NetLink": link between two netnodes
"NetLinkType": type of netlink
"NetElemNode": mapping from net cell to net nodes (counterclockwise)
"NetElemLink": mapping from net cell to its net links (counterclockwise)
"NetLinkContour_x": list of x-contour points of momentum control volume surrounding each net/flow link
"NetLinkContour_y": list of y-contour points of momentum control volume surrounding each net/flow link
"NetLink_xu": x-coordinate of net link center (velocity point)
"NetLink_yu": y-coordinate of net link center (velocity point)
"BndLink": netlinks that co

In [5]:
variable= 'ucx' 
var_data_df= d3d.get_all_data_points(d3d_data, variable, time_index=4)
print(var_data_df)


                   x             y  waterdepth  waterlevel       ucx    time
0      399977.780920  7.161733e+06    0.327972    2.212310 -0.640614  4800.0
1      399987.787225  7.161733e+06    0.337619    2.210880 -0.625273  4800.0
2      399976.977626  7.161749e+06    0.362586    2.212140 -0.680818  4800.0
3      399997.815934  7.161734e+06    0.347271    2.212509 -0.629562  4800.0
4      399986.882145  7.161749e+06    0.366119    2.210620 -0.665612  4800.0
...              ...           ...         ...         ...       ...     ...
11825  401212.479592  7.160735e+06    3.251445    2.397935 -0.118438  4800.0
11826  401217.019308  7.160752e+06    2.625905    2.399721  0.026965  4800.0
11827  401218.693149  7.160722e+06    3.342938    2.391239 -0.065090  4800.0
11828  401222.158795  7.160740e+06    2.802202    2.396301  0.072432  4800.0
11829  401228.105738  7.160727e+06    2.718100    2.393640 -0.010666  4800.0

[11830 rows x 6 columns]


In [6]:
xmin=var_data_df.x.max()
xmax=var_data_df.x.min()

ymin=var_data_df.y.max()
ymax=var_data_df.y.min()

zmin=var_data_df.waterdepth.max()
zmax=var_data_df.waterdepth.min()

In [7]:
Lat_long=open('lat_long.txt')
lat_long=Lat_long.readlines()
Lat= lat_long[0:579]
Long= lat_long[581:1160]
Lat = np.array(Lat)
Lat=Lat.astype(float)
Long = np.array(Long)
Long=Long.astype(float)



FileNotFoundError: [Errno 2] No such file or directory: 'lat_long.txt'

In [ ]:
x_lat= np.tile(Lat, 100)
y_long= np.tile(Long, 100)
z_samples =np.repeat( np.linspace(zmin,zmax, num=100),579)
Point={'x': x_lat, 'y': y_long, 'waterdepth': z_samples}
points=pd.DataFrame(Point)
points

In [ ]:
turbine_x_loc= 400993.57467338
turbine_y_loc= 7161237.67096936
turbine_diameter= 0.7
N=1
y_sample = turbine_y_loc+N*turbine_diameter
x_samples = np.linspace(turbine_x_loc-60, turbine_x_loc+60, num=40)
z_samples = np.linspace(zmin,zmax, num=256)

variables= ['ucy']
sample_points = d3d.create_points(x_samples, y_sample, z_samples) 

Var_sample= d3d.variable_interpolation(d3d_data, variables, points= points)

# Creating new plot limits 
max_plot_TI=27
min_plot_TI=0


# Plotting 
plt.figure(figsize=(10,4.4))
contour_plot = plt.tricontourf(
    Var_sample.x, 
    -Var_sample.waterdepth, 
    Var_sample.ucy,

)

plt.xlabel('x (m)')
plt.ylabel('Water Depth (m)')
plt.title('Velocity in the Y direction')
cbar= plt.colorbar(contour_plot)
cbar.set_label('velocity [m/s]')

In [ ]:
sample_points

In [ ]:
np.tile([1, 2,3,4],3)

In [ ]:
plt.scatter(var_data_df.x,var_data_df.y,var_data_df.waterdepth)
plt.scatter(Lat, Long)

In [ ]:
np.size(Lat)

In [ ]:
np.size(Long)